In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import pickle
import numpy as np


In [2]:
Fulldf = pd.read_csv('waste_data.csv')

In [3]:
Fulldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24108 entries, 0 to 24107
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   date               24108 non-null  int64
 1   month              24108 non-null  int64
 2   year               24108 non-null  int64
 3   Load_Type          24108 non-null  int64
 4   Total_Load_Weight  24108 non-null  int64
dtypes: int64(5)
memory usage: 941.8 KB


In [4]:
Fulldf.isnull().sum()
Fulldf = Fulldf.dropna()
Fulldf.describe(include='all')

,date,month,year,Load_Type,Total_Load_Weight
count,24108.000000,24108.000000,24108.000000,24108.000000,24108.000000
mean,15.730463,6.522648,2015.000747,3.500000,29.951178
std,8.800623,3.448852,3.162540,1.707861,15.789526
min,1.000000,1.000000,2010.000000,1.000000,8.000000
25%,8.000000,4.000000,2012.000000,2.000000,18.000000
50%,16.000000,7.000000,2015.000000,3.500000,26.000000
75%,23.000000,10.000000,2018.000000,5.000000,36.000000
max,31.000000,12.000000,2020.000000,6.000000,85.000000


In [5]:
Fulldf.head()

,date,month,year,Load_Type,Total_Load_Weight
0,1,1,2010,1,32
1,2,1,2010,1,35
2,3,1,2010,1,34
3,4,1,2010,1,34
4,5,1,2010,1,26


In [6]:
Fulldf = pd.DataFrame(Fulldf)


In [7]:
Fulldf.corr()

,date,month,year,Load_Type,Total_Load_Weight
date,1.000000e+00,1.039798e-02,3.559882e-04,-1.695122e-15,-0.000317
month,1.039798e-02,1.000000e+00,-3.096076e-04,-2.920253e-15,-0.002441
year,3.559882e-04,-3.096076e-04,1.000000e+00,2.948351e-12,0.361733
Load_Type,-1.695122e-15,-2.920253e-15,2.948351e-12,1.000000e+00,-0.643129
Total_Load_Weight,-3.174006e-04,-2.440532e-03,3.617328e-01,-6.431294e-01,1.000000


In [8]:
X = Fulldf.drop('Total_Load_Weight', axis=1).values
Y = Fulldf['Total_Load_Weight'].values


In [9]:
x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.2)


In [10]:
model=DecisionTreeClassifier(criterion='gini' 
                            , max_depth= 20
                            , max_features='auto' 
                            , min_samples_leaf= 2
                            , min_samples_split=2)
model.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=100, max_features='auto', min_samples_split=5)

In [17]:
# Writing different model files to file
with open( 'wasteModel.sav', 'wb') as f:
    pickle.dump(model,f)

In [10]:
year = int(input("Input Year: "))
date = input("Input date ")
month = input("Input Month : ")
ty = str(input("input type in block capitals: "))
prediction = model.predict([[date , month , year , ty]])
print(prediction)

[19]


d:\pyenv\penv\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


# Hyper parameter Tuning

In [13]:
#create the data
X = Fulldf.drop('Total_Load_Weight', axis=1).values# Input features (attributes)
y = Fulldf['Total_Load_Weight'].values # Target vector
print('X shape: {}'.format(np.shape(X)))
print('y shape: {}'.format(np.shape(y)))

X shape: (24108, 4)
y shape: (24108,)


In [14]:
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [10,20]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]

In [15]:
# Create the param grid
param_grid = { 'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(param_grid)

{'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2]}


In [16]:
#Instantiate and fit model
model = DecisionTreeClassifier()

In [18]:
rf_Grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 20, verbose=2, n_jobs = 4)

In [19]:
rf_Grid.fit(x_train, y_train)

Fitting 20 folds for each of 16 candidates, totalling 320 fits


d:\pyenv\penv\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 12 members, which is less than n_splits=20.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=20, estimator=DecisionTreeClassifier(), n_jobs=4,
             param_grid={'max_depth': [10, 20],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5]},
             verbose=2)

In [20]:
rf_Grid.best_params_

{'max_depth': 20,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 2}